In [4]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

In [5]:
from sklearn.datasets import fetch_openml

mnist = fetch_openml('mnist_784')

X = mnist.data
y = mnist.target

/Users/ivanosipchyk/dev/studies/chapter_7/venv/lib/python3.11/site-packages/sklearn/datasets/_openml.py:968: FutureWarning: The default value of `parser` will change from `'liac-arff'` to `'auto'` in 1.4. You can set `parser='auto'` to silence this warning. Therefore, an `ImportError` will be raised from 1.4 if the dataset is dense and pandas is not installed. Note that the pandas parser may return different data types. See the Notes Section in fetch_openml's API doc for details.
  warn(


In [6]:
from sklearn.model_selection import train_test_split

X_train_val, X_test, y_train_val, y_test = train_test_split(X, y, test_size=10000, random_state=42)
X_train, X_val, y_train, y_val = train_test_split(X_train_val, y_train_val, test_size=10000, random_state=42)

In [7]:
from sklearn.preprocessing import StandardScaler

scaler = StandardScaler()
X_train_scaled = scaler.fit_transform(X_train)
X_val_scaled = scaler.transform(X_val)
X_test_scaled = scaler.transform(X_test)

In [23]:
from sklearn.ensemble import RandomForestClassifier, ExtraTreesClassifier
from sklearn.svm import SVC

rfc = RandomForestClassifier(n_estimators=100, random_state=42)
extc = ExtraTreesClassifier(n_estimators=100, random_state=42)
svc = SVC(probability=True, random_state=42)

clf_list = [rfc, extc, svc]

for clf in clf_list:
    clf.fit(X_train_scaled, y_train)
    print(clf.__class__.__name__, " is trained.")

RandomForestClassifier  is trained.
ExtraTreesClassifier  is trained.
SVC  is trained.


In [24]:

from sklearn.metrics import accuracy_score

scores = []

for clf in clf_list:
    y_pred = clf.predict(X_val_scaled)
    scores.append(accuracy_score(y_val, y_pred))

scores

[0.9692, 0.9715, 0.9632]

In [19]:
# predicted_probas = []

# for clf in clf_list:
#     predicted_probas.append(clf.predict_proba(X_val_scaled))

# predicted_probas = np.array(predicted_probas)

(1, 784)

In [ ]:
# mean_probas = np.mean([inner_arr for inner_arr in predicted_probas], axis=0)

In [25]:
from sklearn.ensemble import VotingClassifier

estimators = [
    ('random_forest_clf', rfc),
    ('extra_tree_clf', extc),
    ('svm_clf', svc)
]

vtc = VotingClassifier(estimators=estimators, voting='hard')
vtc.fit(X_train_scaled, y_train)

VotingClassifier(estimators=[('random_forest_clf',
                              RandomForestClassifier(random_state=42)),
                             ('extra_tree_clf',
                              ExtraTreesClassifier(random_state=42)),
                             ('svm_clf',
                              SVC(probability=True, random_state=42))])

In [26]:
accuracy_score(vtc.predict(X_val_scaled), y_val)

0.9731

In [27]:
vtc.voting = 'soft'
accuracy_score(vtc.predict(X_val_scaled), y_val)

0.9754

In [28]:
accuracy_score(vtc.predict(X_test_scaled), y_test)

0.9729

# Stacking

In [60]:
X_val_predictions = np.empty((len(X_val), len(clf_list)), dtype=np.float32)

for index, clf in enumerate(clf_list):
    X_val_predictions[:, index] = clf.predict(X_val_scaled)

In [61]:
rnd_forest_blender = RandomForestClassifier(n_estimators=200, oob_score=True, random_state=42)
rnd_forest_blender.fit(X_val_predictions, y_val)

RandomForestClassifier(n_estimators=200, oob_score=True, random_state=42)

In [62]:
rnd_forest_blender.oob_score_

0.9702

In [63]:
X_test_predictions = np.zeros((len(X_test), len(clf_list)), dtype=np.float32)
for index, clf in enumerate(clf_list):
    X_test_predictions[:, index] = clf.predict(X_test_scaled)

In [64]:
y_pred = rnd_forest_blender.predict(X_test_predictions)
accuracy_score(y_test, y_pred)

0.9674

Best model is Voting Classifier with soft voting